<a href="https://colab.research.google.com/github/romash23/project-3/blob/master/BaseLine_by_romash23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sf_booking_path = kagglehub.competition_download('sf-booking')

print('Data source import complete.')


 <h1 style="text-align:center; font-style:italic;">Введение</h1>




<div class="alert alert-info" role="alert">
<center><img src="https://upload.wikimedia.org/wikipedia/commons/e/e3/Booking.com_logo2.png" style="width:200px;height:40px;">
    

### *Какой кейс решаем ?*

Представим, что работаем дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.
    
<center>
    <b>Описание данных</b>
    
    
<table >
   <tr>
    <th>Признак</th>
    <th>Описание</th>
  </tr>
  
   <tr>
    <td>hotel_address</td>
    <td>адрес отеля</td>
  </tr>
    
   <tr>
    <td>additional_number_of_scoring</td>
    <td>количество оценок без отзывов</td>
  </tr>
    
  <tr>
    <td>review_date</td>
    <td>дата отзыва</td>
  </tr>
    
  <tr>
    <td>average_score</td>
    <td>средний балл отеля</td>
  </tr>
    
   <tr>
    <td>hotel_name</td>
    <td>название</td>
  </tr>
    
   <tr>
    <td>reviewer_nationality</td>
    <td>гражданство рецензента</td>
  </tr>
    
   <tr>
    <td>negative_review </td>
    <td>негативный отзыв</td>
  </tr>
   
   <tr>
    <td>review_total_negative_word_counts </td>
    <td>число слов в негативном отзыве</td>
  </tr>  
    
   <tr>
    <td>total_number_of_reviews </td>
    <td>общее число отзывов на странице отеля</td>
  </tr>  
    
   <tr>
    <td>positive_review</td>
    <td>позитивный отзыв</td>
  </tr>
    
   <tr>
    <td>review_total_positive_word_counts</td>
    <td>число слов в позитивном отзыве</td>
  </tr>
    
  <tr>
    <td>total_number_of_reviews_reviewer_has_given</td>
    <td>общее число отзывов рецензента на площадке</td>
  </tr>
    
   <tr>
    <td>reviewer_score</td>
    <td>оценка данная рецензентом отелю</td>
  </tr>
    
   <tr>
    <td>tags</td>
    <td>теги </td>
  </tr>
    
   <tr>
    <td>days_since_review</td>
    <td>дней с момента публикации отзыва</td>
  </tr>
    
   <tr>
    <td>lat</td>
    <td>координата по широте</td>
       
   <tr>
    <td>lng</td>
    <td>координата по долготе</td>
  </tr>
    
</table>
    
</div>

In [ ]:
# Стандартные библиотеки для работы с таблицами и вычислениями.
import numpy as np
import pandas as pd

# Инструменты визуализации.
import matplotlib.pyplot as plt
import seaborn as sns

# Библиотека для кодировки признаков и счетчик хешируемых объектов.
import category_encoders as ce
from collections import Counter

# Метрики и ножницы для рботы с выборками.
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif, chi2

# Модели.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import lightgbm as lgb

# Делаем визуальную настройку графиков
sns.set_palette("Set2")
sns.set_theme(style="dark")

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

## 1. Обзор и очистка данных

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
# Находим дубликаты и удаляем их
all_columns = list(df_train.columns)
mask_dupl = df_train.duplicated(subset=all_columns)
data_duplicates = df_train[mask_dupl]
print(f'Количество найденных дубликатов: {data_duplicates.shape[0]}')
df_train.drop_duplicates(inplace=True,ignore_index=True)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, поэтому пока что заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.nunique(dropna=False)

In [ ]:
data.describe(include='object')

## 2. Обработка данных

### 2.1. Разбор адреса и гражданства

In [ ]:
# Создадим признак - страна отеля
data['hotel_country'] = data['hotel_address'].apply(lambda x: x.split()[-1])

# Для Великобритании
data.loc[data['hotel_country'] == 'Kingdom', 'hotel_country'] = 'United Kingdom'

# Посмотрим, отели каких стран у нас представлены.
data['hotel_country'].value_counts()

In [ ]:
# Создадим признак - город отеля
data['hotel_city'] = data['hotel_address'].apply(lambda x: x.split()[-5] if x.split()[-1] == 'Kingdom' else x.split()[-2])

# Посмортим на города.
data['hotel_city'].value_counts()

In [ ]:
# Cоздадим сводную таблицу для дальнейшей визуализации.
cities_pt = pd.pivot_table(data=data[data['sample']==1], index='hotel_city', values='reviewer_score', aggfunc=['mean'])

figure, ax = plt.subplots(figsize=(10,4))
ax.set_ylim(8, 8.6);
ax.tick_params(axis='x', rotation=45)
sns.barplot(x=cities_pt.index, y=cities_pt[('mean', 'reviewer_score')]);
ax.set_xlabel('Город')
ax.set_ylabel('Средняя оценка')
ax.grid()
plt.title('Средняя оценка отелей по городам', fontsize=15, pad=20);

- Наибольшая средняя оценка отелей наблюдается в Барселоне и Вене. В Лондоне - наименьшая, но и отзывов о лондонских отелях у нас больше всего.

In [ ]:
# Проверим датасет на наличие пропусков:
colors = ['lightblue', 'green']
fig = plt.figure(figsize=(8, 8))
cols = data.columns
ax = sns.heatmap(
    data[cols].isnull()*100,
    cmap=sns.color_palette(colors),
)
ax.set_title("Тепловая карта пропусков данных");

Пропуски в нашем датасете притствуют только в координатах *lat* и *lng*.

In [ ]:
# Посмотрим в какиx городах есть пропуски
data.loc[data['lat'].isnull()]['hotel_city'].value_counts()

Так как библиотека *geopy* рабоает на Kaggle не совсем корректно, то я нашел координаты городов в пропусками на локальной машине. Поскольку *geopy* не смог идентифицировать все адреса из датасета, то чтобы не осталось пропусков, заполним их координатами центра текущего города.

In [ ]:
# Зададим словарь с координатами

city_coords = {
    'Vienna':      (48.210033, 16.363449),
    'Paris':       (48.864716, 2.349014),
    'Barcelona':   (41.38879, 2.15899)
}

In [ ]:
# Функция для заполнения пропусков
def fill_coords(row):
    if np.isnan(row['lat']) or np.isnan(row['lng']):
        coords = city_coords.get(row['hotel_city'])
        if coords:
            if np.isnan(row['lat']):
                row['lat'] = coords[0]
            if np.isnan(row['lng']):
                row['lng'] = coords[1]
    return row

# Применяем функцию к DataFrame
data = data.apply(fill_coords, axis=1)

 ### Гражданство ***reviewer_nationality***

In [ ]:
data['reviewer_nationality'].value_counts()

In [ ]:
# Создаем признак - является ли посетитель отеля гражданином страны, в которой расположен отель
data['is_citizen']=data.apply(lambda x: 1 if x['reviewer_nationality'].strip()==x['hotel_country'] else 0, axis=1)

# Разделим гражданство на категории
def nationality(args):
    if 'United Kingdom' in args:
        return 'Br'
    if 'United States of America' in args:
        return 'Am'
    else:
        return 'other'

data['nationality']=data['reviewer_nationality'].apply(nationality)

In [ ]:
# Закодируем hotel_city и nationality
data = pd.get_dummies(data, columns=['hotel_city','nationality'])
# Удалим лишние признаки
data.drop([ 'nationality_other','reviewer_nationality', 'hotel_country','hotel_address'], axis=1, inplace=True)

In [ ]:
data.info()

### 2.2. Разбор даты

In [ ]:
# Сперва изменим тип представленных в столбце данных.
data['review_date'] = pd.to_datetime(data['review_date'])

In [ ]:
# Разделим даты на кварталы
data['review_quarter'] = data['review_date'].dt.quarter

In [ ]:
# Визуализируем зависимости количества отзывов и средних оценок от сезона.
quarter = data[data['sample']==1].pivot_table(values='reviewer_score', index='review_quarter', aggfunc=['mean','count'])

fig,axes = plt.subplots(1, 2, figsize=(15, 4))

sns.barplot(x=quarter.index, y=quarter[('count', 'reviewer_score')], ax=axes[0], color='green')
axes[0].set_ylabel('Количество отзывов', fontsize = 12)
axes[0].tick_params(axis='x', rotation=75)
axes[0].set_xlabel('Квартал',  fontsize = 12)
axes[0].tick_params(axis='both', which='major', labelsize=9)
axes[0].grid();

sns.barplot(x=quarter.index, y=quarter[('mean', 'reviewer_score')], ax=axes[1], color='lightgreen')
axes[1].set_ylabel('Средняя оценка', fontsize = 12);
axes[1].tick_params(axis='x', rotation=75)
axes[1].set_xlabel('Квартал', fontsize = 12)
axes[1].tick_params(axis='both', which='major', labelsize=9)
plt.ylim(8, 8.7)
axes[1].grid();

- Наибольшее количество отзывов наблюдается в 3 квартале, то есть с июля по сентябрь. Это может быть связано с периодом отпусков, поэтому отели в этот сезон имеют большой спрос.
- Средняя оценка по кварталам немного отличается. Вероятно признак с номером квартала будет полезен при машинном обучении

In [ ]:
# Разделим даты по годам
data['year'] = data['review_date'].dt.year

In [ ]:
# Визуализируем зависимости количества отзывов и средних оценок от сезона.
quarter = data[data['sample']==1].pivot_table(values='reviewer_score', index='year', aggfunc=['mean','count'])

fig,axes = plt.subplots(1, 2, figsize=(15, 4))

sns.barplot(x=quarter.index, y=quarter[('count', 'reviewer_score')], ax=axes[0], color='blue')
axes[0].set_ylabel('Количество отзывов', fontsize = 14)
axes[0].set_xlabel('Год',  fontsize = 14)
axes[0].tick_params(axis='both', which='major', labelsize=9)
axes[0].grid();

sns.barplot(x=quarter.index, y=quarter[('mean', 'reviewer_score')], ax=axes[1], color='lightblue')
axes[1].set_ylabel('Средняя оценка', fontsize = 14);
axes[1].set_xlabel('Год', fontsize = 14)
axes[1].tick_params(axis='both', which='major', labelsize=9)
plt.ylim(8, 8.7)
axes[1].grid();

- В данных виден явный перекос в 2016 году по количеству отзывов
- Признак года также влияет и на среднюю оценку пользователя

In [ ]:
# Закодируем признак года:

year_mapping = {2015: 0, 2016: 1, 2017: 2}
data['year'] = data['year'].map(year_mapping)

In [ ]:
# Удалим обработанный признак.
data.drop(['review_date'], axis=1, inplace=True)

In [ ]:
# Берем из days_since_review количество дней
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]))
correlation = data['days_since_review'].corr(data['reviewer_score'])
print(f'Корреляция между days_since_review и reviewer_score: {correlation:.5f}')


Поскольку корреляция рейтинга отелей и признака *days_since_review* очень мала и совсем непонятно, для чего этот признак нужен, то следует избавиться от него

In [ ]:
# Удалим неинформативный признак
data.drop(['days_since_review'], axis=1, inplace=True)

### 2.3. Количество оценок и отзывов

In [ ]:
fig = plt.figure(figsize=(12,10))
numeric_data = data.select_dtypes(include=['number'])
corr_matrix = numeric_data.corr()

sns.heatmap(corr_matrix, annot=True, fmt='.2f')

### 2.4. Обработка тегов

In [ ]:
# Посчитаем и взглянем на топовые теги.
tags_cntr = Counter(data['tags'].explode())
tags = pd.Series(tags_cntr)
tags.sort_values(ascending=False)[:15]

In [ ]:
# Напишем функцию, преобразующую строковые значения в ячейках столбца
# с тегами и возвращающую их в форме списка.
def func_for_tags(string):
    string = string[3:-3]
    result = list(string.split(" ', ' "))
    return result

data['tags'] = data['tags'].apply(func_for_tags)


print(f"Число уникальных тегов: {len(data['tags'].explode().unique())}")

In [ ]:
# Создадим признак определяющий тип поездки Business или Leisure trip
def trip(line: list) -> int:
    if 'Business trip' in line:
        return 'Командировка'
    if 'Leisure trip' in line:
        return 'Отдых'
    else:
        return 'Отсутствует'

data['trip']=data['tags'].apply(trip)


# Создадим признак определяющий состав заселяющихся в номер.
def comp_checker(line: list) -> int:
    if 'Solo traveler' in line:
        return 1
    if 'Couple' in line:
        return 2
    if 'Group' in line or 'Travelers with friends' in line:
        return 3
    if 'Family with young children' in line or 'Family with older children' in line:
        return 4

data['company']=data['tags'].apply(comp_checker)

# Создадим признак - отправил ли рецензент свой отзыв через телефон.
data['submit'] = data['tags'].apply(lambda x: 1 if 'Submitted from a mobile device' in x else 0)

In [ ]:
trip_types_pivot = pd.pivot_table(data=data[data['sample']==1],
    index='trip', values='reviewer_score', aggfunc=['mean'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

trip_types_count = data['trip'].value_counts()
ax1.pie(trip_types_count, labels=['Отдых', 'Командировка','Отсутствует'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Тип поездки', fontsize=14)

sns.barplot(x=trip_types_pivot.index, y=trip_types_pivot[('mean', 'reviewer_score')], ax=ax2)
ax2.set_title('Зависимость среднего рейтинга от типа поездки',  fontsize=14,pad=20)
ax2.set_ylabel('Средний рейтинг', fontsize=12)
ax2.set_xlabel('Тип поездки', fontsize=12)
plt.ylim(7.5, 9);

Наблюдается зависимость между типом поездки и рейтингом отеля :
- Преобладающая цель поездки-отдых (81%).
- Люди, приехавшие отдохнуть, в среднем ставят оценку выше, чем те, кто в командировке.

In [ ]:
# Сделаем признак количество ночей:

import re

def extract_count_night(string):
    regex = r'Stayed (\d+) night[s]?'
    string = str(string)
    result = re.search(regex, string)
    if result is None:
        return 0
    else:
        return int(result.group(1))

data['night'] = data['tags'].map(extract_count_night)
data['night'].value_counts().head()

In [ ]:
comp_type_pivot = pd.pivot_table(data=data[data['sample']==1],
    index='company', values='reviewer_score', aggfunc=['mean'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

com_types_count = data['company'].value_counts()
ax1.pie(com_types_count, labels=['Пара', 'Одинокий', 'Семья', 'Группа'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Тип пользователя', fontsize=14)

sns.barplot(x=comp_type_pivot.index, y=comp_type_pivot[('mean', 'reviewer_score')], ax=ax2, order=[2,1,4,3])
ax2.set_title('Зависимость среднего рейтинга от типа пользователя',  fontsize=14, pad=20)
ax2.set_ylabel('Средний рейтинг', fontsize=12)
ax2.set_xlabel('Тип пользователя', fontsize=12)
plt.ylim(8, 9);

Наблюдается явное влияние окружения пользователя на оценку, которую он выставляет, а именно:
- Половина пользователей предпочитает отдыхать вдвоем и ставит наибольший рейтинг.
- Высокий балл ставят люди , отдыхающие в группе.
- Одиноким людям обычно все не нравятся, и они ставят низкий балл.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
sns.countplot(x=data['night'], color='lightgreen')
ax.set_title('Распределение числа ночей проведенных в номерах отелей',  size=15,pad=20)
ax.set_ylabel('Частота', fontsize = 12, labelpad=10)
ax.set_xlabel('Число ночей', fontsize = 12,labelpad=10);

- Наблюдается равномерное падение числа ночей проведенных в отеле.
- Большинство отелей рассматривается с целью переночевать на одну ночь, а не отдохнуть на несколько дней.

In [ ]:
pivot_nights = pd.pivot_table(data=data[data['sample']==1],
    index='night', values='reviewer_score', aggfunc=['mean'])

fig1, ax1 = plt.subplots(figsize=(15, 4))
plt.suptitle('Средняя оценка в завивисимости от кол-ва проведенных ночей', size=15)
sns.barplot(x=pivot_nights.index, y=pivot_nights[('mean', 'reviewer_score')], color='lightblue')
plt.xlabel('Количество проведенных ночей', fontsize = 12)
plt.ylabel('Средний отзыв', fontsize = 12)
plt.ylim(7, 9.5);

Явной зависимости средней оценки в зависимости от количества дней - нет. Но после 13го дня отличия налицо, есть вероятность, что данный признак повлияет на улучшение предсказания.

In [ ]:
smd_from_mob_pivot = pd.pivot_table(data=data[data['sample']==1],
    index='submit', values='reviewer_score', aggfunc=['mean'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

smd_type = data['submit'].value_counts()
ax1.pie(smd_type, labels=['с телефона', 'с компьютера'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Отзыв отправлен', fontsize=14)

sns.barplot(x=smd_from_mob_pivot.index, y=smd_from_mob_pivot[('mean', 'reviewer_score')], ax=ax2, order=[1, 0])
ax2.set_title('Зависимость среднего рейтинга\nот способа отправки отзыва',  fontsize=14)
ax2.set_ylabel('Средний рейтинг', fontsize=12)
ax2.set_xlabel('Отзыв отправлен', fontsize=12)
ax2.grid()
plt.ylim(8, 9);

- Средний рейтинг у пользователей, которые отправили отзыв с телефона, несущественно выше.

In [ ]:
# Удалим обработанный признак.
data.drop(['tags'], axis=1, inplace=True)

In [ ]:
# Закодируем trip,company
data = pd.get_dummies(data, columns=['trip','company'])

# Удалим лишние признаки
#data.drop(['submit'], axis=1, inplace=True)


In [ ]:
# Удалим неинформативный признак
data.drop(['trip_Отсутствует'], axis=1, inplace=True)

### 2.5. Обработка отзывов

In [ ]:
data['negative_review'].value_counts()

In [ ]:
# Создадим признак определяющий есть ли негативный отзыв

def negative_normalize(text):

    negative_variants = [
        '', 'n a', 'na', 'none', 'nada', ' ', 'loved everything',
        'nil', 'can t think of anything', 'everything was perfect',
        'everything was great', 'all good', 'leaving', 'see above',
        'non', 'everything was good', 'i liked everything', 'we liked everything',
        'all was good', 'liked everything', 'as above', 'everything was fine',
        'liked it all', 'there wasn t anything we didn t like', 'all great'
    ]

    text_clean = text.strip().lower()

    # Убираем лишние пробелы внутри строки

    text_clean = ' '.join(text_clean.split())
    text_set = set(text_clean.split())

    if text_clean in negative_variants or text_set.intersection(['nothing', 'no', 'not', 'good', 'ok']):
        return 0 # 0 - негативный отзыв отсутствует
    else:
        return 1 # 1 - значит отзыв действительно негативный

data['negative_binary'] = data['negative_review'].map(negative_normalize)
data['negative_binary'].value_counts()

In [ ]:
data['negative_binary'].value_counts()

In [ ]:
data['positive_review'].value_counts()

In [ ]:
# Создадим признак определяющий позитивный отзыв

def pozitive_normalize(text):

    text_clean = text.strip().lower()

    # Убираем лишние пробелы внутри строки
    text_clean = ' '.join(text_clean.split())
    text_set = set(text_clean.split())

    if text_set.intersection(['nothing', 'no', 'not', 'bad', 'upset', 'awful']):
        return 0 # В позитивном отзыве либо пусто, либо негативные слова
    else:
        return 1 # Отзыв дествительно позитивный

data['positive_binary'] = data['positive_review'].map(pozitive_normalize)
data['positive_binary'].value_counts()

In [ ]:
data['positive_binary'].value_counts()

In [ ]:
# Удалим лишние признаки
data.drop(['positive_review','negative_review'], axis=1, inplace=True)

### 2.6 Наименование отеля

In [ ]:
data['hotel_name'].value_counts()

In [ ]:
# Создадим признак определяющий позитивный отзыв
def hotel_name(args):
    if 'Britannia International Hotel Canary Wharf' in args or 'Strand Palace Hotel' in args or 'Park Plaza Westminster Bridge London' in args :
        return 1
    if 'Copthorne Tara Hotel London Kensington' in args or 'DoubleTree by Hilton Hotel London Tower of London' in args:
        return 2
    else:
        return 0

data['name']=data['hotel_name'].apply(hotel_name)

In [ ]:
data['name'].value_counts()

In [ ]:
# Закодируем name
data = pd.get_dummies(data, columns=['name'])

In [ ]:
# Удалим лишние признаки
data.drop(['name_0'], axis=1, inplace=True)

In [ ]:
# Удалим лишние признаки
data.drop(['hotel_name'], axis=1, inplace=True)

## 3. Корреляция признаков

In [ ]:
plt.rcParams['figure.figsize'] = (22,16)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True, fmt='.2f')

In [ ]:
# Удалим признаки с сильной корреляционной связью:
data.drop(['hotel_city_Barcelona', 'trip_Командировка'], axis=1, inplace=True)

## 4. Нормализация данных

Проверим числовые признаки на нормальность. Поскольку тест Шапиро-Уилка предназначен для небольших и средних выборок (до 5000 наблюдени), то воспользуемся тестом Андерсона-Дарлинга

In [ ]:
import scipy.stats as stats

# Проверка нормальности через тест Андерсона-Дарлинга
def anderson_darling_test(data, alpha=0.05):

    result = stats.anderson(data, dist='norm')
    print(f"Anderson-Darling: A^2 = {result.statistic:.3f}")

    # Сравниваем с критическими значениями для выбранного уровня значимости
    critical_values = result.critical_values
    significance_levels = result.significance_level

    # Находим индекс для заданного alpha (обычно 5% = 0.05)
    idx = np.where(significance_levels == alpha*100)[0][0]

    if result.statistic > critical_values[idx]:
        print(f"Распределение не нормальное (AD-test на уровне {alpha})")
    else:
        print(f"Распределение нормальное (AD-test на уровне {alpha})")

In [ ]:
# В переменную digit_cols запишем стоблцы, которые мы будет проверять на нормальность
digit_cols = list(data.nunique()[data.nunique() > 4].index)
for col in digit_cols:
    print(col,':')
    anderson_darling_test(data[col])
    print('-'*40)

#### Все рассматриваемые признаки не распределены нормально, поэтому займемся их нормализацией. Поскольку мы не удаляли выбросы, оптимально будет использовать *RobustScaler*

In [ ]:
from sklearn.preprocessing import RobustScaler

if 'reviewer_score' in digit_cols:
    digit_cols.remove('reviewer_score')

data[digit_cols] = data[digit_cols].astype(float)
data[digit_cols] = RobustScaler().fit_transform(data[digit_cols].values)


## 5. Разделение выборки на *x* и *y*, обучение модели

In [ ]:
# убираем признаки которые еще не успели обработать,
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
from sklearn.feature_selection import f_classif # anova

columns=X.columns

imp_num = pd.Series(f_classif(X, y)[0], index = columns)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh');

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh');

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)